In [ ]:
import os
import sys
root_dir = os.path.join(os.getcwd(), '../../')
if root_dir not in sys.path:
    sys.path.append(root_dir)
from spider_local import SPIDER
%reload_ext autoreload
%autoreload 2
op = SPIDER()
import anndata
import scanpy as sc
import pandas as pd
import numpy as np


In [ ]:
adata = anndata.read_h5ad('./adata.h5ad')
adata

In [ ]:
out_f # where to put the results
R_path # path to your R executor

In [ ]:
idata = op.prep(adata, out_f, R_path, cluster_key='cluster', is_human=True, coord_type='grid', no_spatalk=False)

In [ ]:
idata, meta_idata = op.find_svi(idata, out_f, R_path, alpha=0.3)

In [ ]:
svi_df, svi_df_strict = op.svi.combine_SVI(idata,threshold=0.01)
op.svi.eva_SVI(idata, svi_df_strict)

In [ ]:
op.vis.pattern_LRI(idata,show_SVI=10)
plt.tight_layout()

In [ ]:
adata_copy = adata.copy()
sc.pp.normalize_total(adata_copy, target_sum=1e4)
sc.pp.log1p(adata_copy)
sc.pp.highly_variable_genes(adata_copy, flavor='seurat_v3', n_top_genes=5000)
genes = adata_copy[:, adata_copy.var.highly_variable].var_names.values[:5000]

sq.gr.spatial_neighbors(adata, key_added='spatial')
sq.gr.spatial_autocorr(
    adata,
    genes=genes,
    mode="moran",
    n_perms=1000,
    n_jobs=10,
)
merged_df,lri_pw_list,gene_lr_list,gene_pw_list = op.vis.svg_svi_relation(adata, idata, title='PDAC', is_human=False, top=50)


In [ ]:
op.util.interaction_spot_interface(idata)
adata_lri = anndata.AnnData(idata.uns['cell_score'].loc[adata.obs_names])
adata_lri.obs=adata.obs
adata_lri.obsm['spatial'] = adata.obsm['spatial']
st_decon = adata.obsm['deconvolution']
st_decon_org = st_decon.copy()

In [ ]:
decon_corr = pd.concat([st_decon, adata_lri.to_df()], axis=1).corr()[adata_lri.var_names].loc[st_decon.columns].T
decon_corr_sup = decon_corr.loc[decon_corr.idxmax()] 
decon_corr = decon_corr[decon_corr_sup.columns[np.argsort(-decon_corr_sup.to_numpy().diagonal())]]
decon_corr_sup = decon_corr.loc[decon_corr.idxmax()] 

In [ ]:
plt.rcParams.update({'font.size': 18})
st_decon = st_decon_org.copy()
celtypes = decon_corr_sup.columns
st_decon = pd.concat([idata.uns['cell_meta'][['x', 'y']], st_decon], axis=1)
plt.figure(figsize=(len(celtypes)*4, 8))
base = 1
for i in celtypes:
    plt.subplot(2, len(celtypes), base)
    # sns.scatterplot(st_decon, x='x', y='y', hue=i, s=10, linewidth=0)
    im=plt.scatter(st_decon['x'],st_decon['y'], c=st_decon[i], s=50, cmap='plasma')
    plt.colorbar(im,fraction=0.046, pad=0.04)

    plt.axis('equal')
    plt.axis('off')
    plt.title(i.replace('_', ' '))
    base += 1
    
corrs = decon_corr_sup.to_numpy().diagonal()

marker_score = adata_lri.to_df()[decon_corr_sup.index]
marker_score = pd.concat([adata_lri.obs[['x', 'y']], marker_score], axis=1)
for i in range(len(decon_corr_sup.index)):
    plt.subplot(2, len(celtypes), base)
    im=plt.scatter(marker_score['x'],marker_score['y'], c=marker_score[decon_corr_sup.index[i]].to_numpy(), s=50, cmap='plasma')
    # sns.scatterplot(marker_score, x='x', y='y', hue=i, s=10, linewidth=0)
    plt.colorbar(im,fraction=0.046, pad=0.04)
    plt.axis('equal')
    plt.axis('off')
    plt.title(f'{decon_corr_sup.index[i]} \n corr={"%.3f" % corrs[i]}')
    base += 1

plt.tight_layout()